In [1]:
import numpy as np
import math
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

from time import localtime, strftime

%matplotlib inline

some links :<br>
- https://cloud.google.com/ml-engine/docs/scikit/custom-pipeline?hl=fr<br>
- https://towardsdatascience.com/custom-transformers-and-ml-data-pipelines-with-python-20ea2a7adb65<br>
- https://www.kdnuggets.com/2019/10/extend-scikit-learn-bring-sanity-machine-learning-workflow.html

In [3]:
this_building = 493

# {0: electricity, 1: chilledwater, 2: steam, 3: hotwater}
this_meter = 0

data_folder = '../data/raw/csvs/'



In [14]:
def load_data(building_id, meter_id, data_folder_path):
    
    # Loads building_metadata to get site-building mapping
    bdata = pd.read_csv(data_folder_path + 'building_metadata.csv', 
                        index_col='building_id', 
                        usecols=['building_id', 'site_id'])

    this_site = bdata.loc[this_building, 'site_id']
    print('building is on site {}'.format(this_site))
    
    
    # Loads meter_reading data
    raw_df_meters = pd.read_csv(data_folder_path + 'train.csv', parse_dates=['timestamp'])

    to_keep = (raw_df_meters['building_id']==this_building) & (raw_df_meters['meter']==this_meter)
    b_df_meters = raw_df_meters[to_keep].copy()

    b_df_meters.drop('building_id', axis=1, inplace=True)
    b_df_meters.drop('meter', axis=1, inplace=True)

    b_df_meters.set_index('timestamp', inplace=True)
    b_df_meters.sort_index(inplace=True)
    
    # Loads weather data
    
    weather_features_to_load = ['air_temperature', 'dew_temperature'] # TODO as funct arg ?
    
    raw_df_weather = pd.read_csv(
        data_folder_path + 'weather_train.csv', 
        parse_dates=['timestamp'],
        index_col=['site_id','timestamp'],
        usecols=['site_id','timestamp'] + weather_features_to_load
    )

    b_df_weather = raw_df_weather.loc[(this_site,)]

    
    # Joins meter_reading and weather dfs.
    #df_features = b_df_meters.join(b_df_weather, on='timestamp', how='left')
    
    return (b_df_meters, b_df_weather)
    

In [15]:
a, b = load_data(this_building, this_meter, data_folder)

building is on site 3


In [16]:
a.head()

,meter_reading
timestamp,
2016-01-01 00:00:00,284.52
2016-01-01 01:00:00,290.52
2016-01-01 02:00:00,292.80
2016-01-01 03:00:00,289.89
2016-01-01 04:00:00,290.01


In [17]:
b.head()

,air_temperature,dew_temperature
timestamp,,
2016-01-01 00:00:00,10.0,2.2
2016-01-01 01:00:00,9.4,2.8
2016-01-01 02:00:00,8.9,2.2
2016-01-01 03:00:00,7.8,1.1
2016-01-01 04:00:00,7.8,0.6
